In [ ]:
import pyPI as pi
import numpy as np
from scipy.optimize import minimize

In [ ]:
def calculator(In, Out, delta, p=False):

    def objective(params):
        alpha = params[0]
        matrix = pi.jones_matrix(delta, alpha)
        result = matrix @ In
        return np.linalg.norm(result - Out)

    initial_guess = [0]
    result = minimize(objective, initial_guess, bounds=[(0, 2*np.pi)])
    alpha = result.x[0]

    if p:
        print(f"delta:{delta}, alpha:{alpha / np.pi:.4f}π")

    return alpha

In [ ]:
In = pi.polarization_basis_set('P45')
Out = pi.polarization_basis_set('R')
alpha_1, alpha_2 = pi.PSG_calculator(In, Out, np.pi)

In [ ]:
pi.field_notation(pi.PSG(alpha_1, alpha_2) @ In,p=True)